In [11]:
from array import array
import reprlib
import math
import itertools


class Vector2d:
    typecode = 'd'

    def __init__(self, x, y):
        self.__x = float(x)
        self.__y = float(y)

    @property
    def x(self):
        return self.__x

    @property
    def y(self):
        return self.__y

    def __hash__(self):
        return hash(self.x) ^ hash(self.y)

    def __iter__(self):
        return (i for i in (self.__x, self.__y))

    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(array(self.typecode, self)))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __abs__(self):
        return math.hypot(self.__x, self.__y)

    def __bool__(self):
        return bool(abs(self))

    def angle(self):
        return math.atan2(self.__y, self.__x)

    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('p'):
            fmt_spec = fmt_spec[:-1]
            coords = (abs(self), self.angle())
            outer_fmt = '<{}, {}>'
        else:
            coords = self
            outer_fmt = '({}, {})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(*components)

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)


In [2]:
v1 = Vector2d(3, 4)
print(format(v1))
print(format(v1, '.2f'))
print(format(v1, '.3e'))
print(format(v1, 'p'))

(3.0, 4.0)
(3.00, 4.00)
(3.000e+00, 4.000e+00)
<5.0, 0.9272952180016122>


In [3]:
v2 = Vector2d(3.1, 4.2)
print(hash(v1))
print(hash(v2))
print(set([v1, v2]))

7
384307168202284039
{Vector2d(3.1, 4.2), Vector2d(3.0, 4.0)}


In [4]:
v1.__dict__

{'_Vector2d__x': 3.0, '_Vector2d__y': 4.0}

In [5]:
class Vector:
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components)
        print(components)
        components = components[components.find('['):-1]
        print(components)
        return 'Vector({})'.format(components)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))

    def __bool__(self):
        return bool(abs(self))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

In [6]:
v1 = Vector([3, 4, 5])
v1

array('d', [3.0, 4.0, 5.0])
[3.0, 4.0, 5.0]


Vector([3.0, 4.0, 5.0])

In [7]:
a = reprlib.repr(array('d', [1, 2, 3]))
print(type(a))
print(a[a.find('['):-1])
print(array('d', [1, 2, 3]))

<class 'str'>
[1.0, 2.0, 3.0]
array('d', [1.0, 2.0, 3.0])


In [8]:
import collections
Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck2(collections.MutableSequence):
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self) -> None:
        self._cards = [
            Card(rank, suit) for suit in self.suits for rank in self.suits
        ]

    def __len__(self) -> int:
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]

    def __setitem__(self, position, value):
        self._cards[position] = value

    def __delitem__(self, position):
        del self._cards[position]

    def insert(self, position, value):
        self._cards.insert(position, value)

/var/folders/6b/w3xj46rs3b1dj1fjtmvfbv6w0000gn/T/ipykernel_27974/2194778639.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  class FrenchDeck2(collections.MutableSequence):


In [9]:
import re
import reprlib

RE_WORD = re.compile('\w+')

class Sentence:

    def __init__(self, text) -> None:
        self.text = text
        self.words = RE_WORD.findall(text)

    def __getitem__(self, index):
        return self.words[index]

    def __len__(self):
        return len(self.words)

    def __repr__(self) -> str:
        return 'Setence(%s)' % reprlib.repr(self.text)

    def __iter__(self):
        for word in self.words:
            yield word

In [10]:
s = 'ABC'
it = iter(s)
while True:
    try:
        print(next(it))
    except StopIteration:
        del it
        break

A
B
C


In [12]:
gen = itertools.takewhile(lambda n: n < 3, itertools.count(1, .5))

In [13]:
list(gen)

[1, 1.5, 2.0, 2.5]

## 标准库中的生成器函数

### 用于过滤的生成器函数
1. itertools.compress(it, selector_it) 并行处理两个可迭代的对象；如果 selector_it中的元素是真值，产出 it 中对应的元素
2. itertools.dropwhile(predicate,it) 处理 it，跳过 predicate 的计算结果为真值的元素，然后产出剩下的各个元素（不再进一步检查）
3. filter(predicate, it) 把 it 中的各个元素传给 predicate，如果predicate(item) 返回真值，那么产出对应的元素；如果 predicate 是 None，那么只产出真值元素
4. itertools.filterfalse(predicate,it)
5. itertools.islice(it, stop) 或 itertools.islice(it, start,stop, step=1)
6. itertools.takewhile(predicate,it)

In [25]:
def vowel(c):
    return c.lower() in 'aeiou'
test_str = 'Aardvark'

In [26]:
list(filter(vowel, test_str))

['A', 'a', 'a']

In [27]:
list(itertools.filterfalse(vowel, test_str))

['r', 'd', 'v', 'r', 'k']

In [28]:
list(itertools.dropwhile(vowel, test_str))

['r', 'd', 'v', 'a', 'r', 'k']

In [29]:
list(itertools.takewhile(vowel, test_str))

['A', 'a']

In [30]:
list(itertools.compress(test_str, (1, 0, 1, 1, 0, 1)))

['A', 'r', 'd', 'a']

In [31]:
list(itertools.islice(test_str, 4))

['A', 'a', 'r', 'd']

In [32]:
list(itertools.islice(test_str, 4, 7))

['v', 'a', 'r']

In [33]:
list(itertools.islice(test_str, 1, 7, 2))

['a', 'd', 'a']

### 用于映射的生成器函数

1. itertools.accumulate(it, [func]) 产出累计的总和
2. enumerate(iterable, star=0)
3. map(func, it1, it2, ..., itN)
4. itertools.starmap(func, it)

In [34]:
sample = [5, 4, 2, 8, 7, 6, 3, 0, 9, 1]
list(itertools.accumulate(sample))

[5, 9, 11, 19, 26, 32, 35, 35, 44, 45]

In [35]:
list(itertools.accumulate(sample, min))

[5, 4, 2, 2, 2, 2, 2, 0, 0, 0]

In [36]:
list(itertools.accumulate(sample, max))

[5, 5, 5, 8, 8, 8, 8, 8, 9, 9]

In [37]:
import operator
list(itertools.accumulate(sample, operator.mul))

[5, 20, 40, 320, 2240, 13440, 40320, 0, 0, 0]

In [38]:
list(itertools.accumulate(range(1, 11), operator.mul))

[1, 2, 6, 24, 120, 720, 5040, 40320, 362880, 3628800]

In [50]:
list(enumerate('albatroz', 1))

[(1, 'a'),
 (2, 'l'),
 (3, 'b'),
 (4, 'a'),
 (5, 't'),
 (6, 'r'),
 (7, 'o'),
 (8, 'z')]

In [40]:
list(map(operator.mul, range(11), range(11)))

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100]

In [41]:
list(map(operator.mul, range(11), [2, 4, 8]))

[0, 4, 16]

In [42]:
list(itertools.starmap(operator.mul, enumerate('albatroz', 1)))

['a', 'll', 'bbb', 'aaaa', 'ttttt', 'rrrrrr', 'ooooooo', 'zzzzzzzz']

In [48]:
list(itertools.starmap(lambda a, b: b/a, enumerate(itertools.accumulate(sample), 1)))

[5.0,
 4.5,
 3.6666666666666665,
 4.75,
 5.2,
 5.333333333333333,
 5.0,
 4.375,
 4.888888888888889,
 4.5]

### 合并多个可迭代对象的生成器函数

1. itertools.chain(it, ..., itN)
2. itertools.chain.from_iterable(it)
3. itertools.product(it1, ..., itN, repeat=1)
4. zip
5. itertools.zip_logest(it1, ..., itN, fillvalue=None)

In [51]:
list(itertools.chain('ABC', range(2)))

['A', 'B', 'C', 0, 1]

In [52]:
list(itertools.chain(enumerate('ABC')))

[(0, 'A'), (1, 'B'), (2, 'C')]

In [53]:
list(itertools.chain.from_iterable(enumerate('ABC')))

[0, 'A', 1, 'B', 2, 'C']

In [54]:
list(zip('ABC', range(5)))

[('A', 0), ('B', 1), ('C', 2)]

In [55]:
list(itertools.zip_longest('ABC', range(5)))

[('A', 0), ('B', 1), ('C', 2), (None, 3), (None, 4)]

In [56]:
list(itertools.zip_longest('ABC', range(5), fillvalue='?'))

[('A', 0), ('B', 1), ('C', 2), ('?', 3), ('?', 4)]

In [57]:
list(itertools.product('ABC', range(2)))

[('A', 0), ('A', 1), ('B', 0), ('B', 1), ('C', 0), ('C', 1)]

In [59]:
suits = 'spades hearts diamonds clubs'.split()
list(itertools.product('AK', suits))

[('A', 'spades'),
 ('A', 'hearts'),
 ('A', 'diamonds'),
 ('A', 'clubs'),
 ('K', 'spades'),
 ('K', 'hearts'),
 ('K', 'diamonds'),
 ('K', 'clubs')]

In [60]:
list(itertools.product('ABC'))

[('A',), ('B',), ('C',)]

In [61]:
list(itertools.product('ABC', repeat=2))

[('A', 'A'),
 ('A', 'B'),
 ('A', 'C'),
 ('B', 'A'),
 ('B', 'B'),
 ('B', 'C'),
 ('C', 'A'),
 ('C', 'B'),
 ('C', 'C')]

In [62]:
list(itertools.product(range(2), repeat=3))

[(0, 0, 0),
 (0, 0, 1),
 (0, 1, 0),
 (0, 1, 1),
 (1, 0, 0),
 (1, 0, 1),
 (1, 1, 0),
 (1, 1, 1)]

In [64]:
rows = itertools.product('AB', range(2), repeat=2)
for row in rows:
    print(row)

('A', 0, 'A', 0)
('A', 0, 'A', 1)
('A', 0, 'B', 0)
('A', 0, 'B', 1)
('A', 1, 'A', 0)
('A', 1, 'A', 1)
('A', 1, 'B', 0)
('A', 1, 'B', 1)
('B', 0, 'A', 0)
('B', 0, 'A', 1)
('B', 0, 'B', 0)
('B', 0, 'B', 1)
('B', 1, 'A', 0)
('B', 1, 'A', 1)
('B', 1, 'B', 0)
('B', 1, 'B', 1)


### 把输入的各个元素扩展成多个输出元素的生成器函数

1. itertools.combinations(it, out_len)
2. itertools.combinations_with_replacement(it, out_len)
3. itertools.count(start=0, step=1)
4. itertools.cycle(it)
5. itertools.permutations(it, out_len=None)
6. itertools.repeat(item, [times])

In [65]:
ct = itertools.count()
next(ct)

0

In [66]:
next(ct), next(ct), next(ct)

(1, 2, 3)

In [67]:
list(itertools.islice(itertools.count(1, .3), 3))

[1, 1.3, 1.6]

In [68]:
cy = itertools.cycle('ABC')
list(itertools.islice(cy, 7))

['A', 'B', 'C', 'A', 'B', 'C', 'A']

In [69]:
rp = itertools.repeat(7)
next(rp), next(rp)

(7, 7)

In [70]:
list(itertools.repeat(8, 4))

[8, 8, 8, 8]

In [73]:
list(map(operator.mul, range(11), itertools.repeat(5)))

[0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50]

In [74]:
list(itertools.combinations('ABC', 2))

[('A', 'B'), ('A', 'C'), ('B', 'C')]

In [75]:
list(itertools.combinations_with_replacement('ABC', 2))

[('A', 'A'), ('A', 'B'), ('A', 'C'), ('B', 'B'), ('B', 'C'), ('C', 'C')]

In [76]:
list(itertools.permutations('ABC', 2))

[('A', 'B'), ('A', 'C'), ('B', 'A'), ('B', 'C'), ('C', 'A'), ('C', 'B')]

In [77]:
list(itertools.product('ABC', repeat=2))

[('A', 'A'),
 ('A', 'B'),
 ('A', 'C'),
 ('B', 'A'),
 ('B', 'B'),
 ('B', 'C'),
 ('C', 'A'),
 ('C', 'B'),
 ('C', 'C')]

### 用于重新排列元素的生成器函数]
1. itertools.groupby(it, kye=None)
2. reversed(seq)
3. itertools.tee(it, n=2)

In [78]:
list(itertools.groupby('LLLLAAGGG'))

[('L', <itertools._grouper at 0x12181abe0>),
 ('A', <itertools._grouper at 0x12181a5b0>),
 ('G', <itertools._grouper at 0x120700dc0>)]

In [79]:
for char, group in itertools.groupby('LLLLAAAGG'):
    print(char, '->', list(group))

L -> ['L', 'L', 'L', 'L']
A -> ['A', 'A', 'A']
G -> ['G', 'G']


In [80]:
animals = ['duck', 'eagle', 'rat', 'giraffe', 'bear', 'bat', 'dolphin', 'shark', 'lion']

In [82]:
animals.sort(key=len)
animals

['rat', 'bat', 'duck', 'bear', 'lion', 'eagle', 'shark', 'giraffe', 'dolphin']

In [83]:
for length, group in itertools.groupby(animals, len):
    print(length, '->', list(group))

3 -> ['rat', 'bat']
4 -> ['duck', 'bear', 'lion']
5 -> ['eagle', 'shark']
7 -> ['giraffe', 'dolphin']


In [84]:
for length, group in itertools.groupby(reversed(animals), len):
    print(length, '->', list(group))

7 -> ['dolphin', 'giraffe']
5 -> ['shark', 'eagle']
4 -> ['lion', 'bear', 'duck']
3 -> ['bat', 'rat']


In [86]:
list(itertools.tee('ABC'))
g1, g2 = itertools.tee('ABC')

In [87]:
next(g1)

'A'

In [88]:
next(g2)

'A'

In [89]:
list(zip(*itertools.tee('ABC')))

[('A', 'A'), ('B', 'B'), ('C', 'C')]

In [90]:
all([1, 2, 3])

True

In [91]:
all([1, 0, 3])

False

In [92]:
all([])

True

In [93]:
any([1, 2, 3])

True

In [94]:
any([1, 0, 3])

True

In [95]:
any([0, 0.0])

False

In [104]:
from random import randint

def d6():
    return randint(1, 6)

d6_iter = iter(d6, 1)
d6_iter

In [105]:
for roll in d6_iter:
    print(roll)

2
6
4
5
3
3
2


In [124]:
def fibonacci():
    a, b = 0, 1
    while True:
        yield a
        a, b = b, a + b

In [125]:
fib = fibonacci()
for _ in range(10):
    print(next(fib))

0
1
1
2
3
5
8
13
21
34
